In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import ImageFilter
#画像の読み込み
directory_name_train_0 = '/content/drive/MyDrive/Colab Notebooks/train/dog/'
directory_name_train_1= '/content/drive/MyDrive/Colab Notebooks/train/cat/'


Mounted at /content/drive


In [2]:
label = 0 
label2 =1
number =50
num_words =128
all_bowDescriptors=np.zeros((number*2,num_words),np.float32)
all_labels=np.zeros(number*2,np.int64)
file_name_list_train_0 = [ ] 
file_name_list_train_1 = [ ] 
#BFマッチング用のオブジェクト生成
bf = cv2.BFMatcher( )
#k-meansを行うためのオブジェクトを準備

bowTrainer = cv2.BOWKMeansTrainer(num_words)

kaze = cv2.KAZE_create()


#犬
for i in range(0, number): 
  file_name_train_0 = directory_name_train_0 + 'dog.' + str(i) + '.jpg'
  file_name_list_train_0.append([label, file_name_train_0])
for i in range(0, number):
  label, file_name_train_0 = file_name_list_train_0[i]
  train_0_img = cv2.imread(file_name_train_0)
  #画像をグレーに
  gray_img = cv2.cvtColor(train_0_img, cv2.COLOR_BGR2GRAY)
  #KAZEで局所特徴点(kp)、局所特徴量を取得(des)
  kp, des = kaze.detectAndCompute(gray_img , None)
  #すべての局所特徴量をbowTrainer に格納
  bowTrainer.add( des )

#猫
for i in range(0, number): 
  file_name_train_1 = directory_name_train_1 + 'cat.' + str(i) + '.jpg'
  file_name_list_train_1.append([label2, file_name_train_1])
for i in range(0, number):
  label, file_name_train_1 = file_name_list_train_1[i]
  train_1_img = cv2.imread(file_name_train_1)
  #画像をグレーに
  gray_img = cv2.cvtColor(train_1_img, cv2.COLOR_BGR2GRAY)
  #KAZEで局所特徴点(kp)、局所特徴量を取得(des)
  kp, des = kaze.detectAndCompute(gray_img , None)
  #すべての局所特徴量をbowTrainer に格納
  bowTrainer.add( des )


In [3]:
#すべての局所特徴量をk-means法でクラスタリングし、num_words個のVisualWordを決定する
codebook = bowTrainer.cluster( )
#BoVMを生成するためのオブジェクトを準備
bowExtractor = cv2.BOWImgDescriptorExtractor(kaze, bf)
#bowExtractorにVisual Wordsの辞書をセットする
bowExtractor.setVocabulary(codebook)



In [4]:
#犬
for i in range(0, number):
  label, file_name_train_0 = file_name_list_train_0[i]
  train_0_img = cv2.imread(file_name_train_0)
  #画像をグレーに
  gray_img = cv2.cvtColor(train_0_img, cv2.COLOR_BGR2GRAY)
  #KAZEで局所特徴点(kp)、局所特徴量を取得(des)
  kp, des = kaze.detectAndCompute(gray_img , None)
  #入力画像のBoVWベクトルを生成
  bowDescriptors = bowExtractor.compute(gray_img, kp)
  all_bowDescriptors[i] = bowDescriptors
  all_labels[i] = label

#猫
for j in range(0, number):
  label, file_name_train_1 = file_name_list_train_1[j]
  train_1_img = cv2.imread(file_name_train_1)
  #画像をグレーに
  gray_img = cv2.cvtColor(train_1_img, cv2.COLOR_BGR2GRAY)
  #KAZEで局所特徴点(kp)、局所特徴量を取得(des)
  kp, des = kaze.detectAndCompute(gray_img , None)
  #入力画像のBoVWベクトルを生成
  bowDescriptors = bowExtractor.compute(gray_img, kp)
  all_bowDescriptors[i+j+1] = bowDescriptors
  all_labels[i+j+1] = label


In [5]:
print(all_bowDescriptors)

[[0.00520833 0.01302083 0.00520833 ... 0.00260417 0.05208334 0.        ]
 [0.         0.00646204 0.00807755 ... 0.00484653 0.00484653 0.        ]
 [0.         0.01333333 0.03333334 ... 0.02666667 0.00666667 0.00666667]
 ...
 [0.         0.01226994 0.         ... 0.         0.00613497 0.00613497]
 [0.         0.         0.03529412 ... 0.         0.02352941 0.01176471]
 [0.         0.         0.02542373 ... 0.00847458 0.         0.01694915]]


In [6]:
print(all_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [7]:
#BoVWを行うためのオブジェクトを準備
svm = cv2.ml.SVM_create()
#SVMの設定
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_RBF)
svm.setGamma(1)
svm.setC(1)
svm.setTermCriteria((cv2.TERM_CRITERIA_COUNT, 100, 1.e-06))

print(all_bowDescriptors.shape)
print(all_labels.shape)

#SVM学習の実施
svm.train(all_bowDescriptors, cv2.ml.ROW_SAMPLE, all_labels)
#学習結果の保存
svm.save('/content/drive/MyDrive/Colab Notebooks/train/svm_trained_data.xml')




(100, 128)
(100,)


In [8]:
test= '/content/drive/MyDrive/Colab Notebooks/test/'
label=0
test_num =30
correct_num=0
file_name_list_test = [ ] 
print("犬 = 0 ,猫 = 1")
for i in range(0,test_num):
  if i<=test_num/2-1:
    label=0
  else:
    label=1
  file_name_test = test + 'test.' + str(i) + '.jpg'
  file_name_list_test.append([label, file_name_test])

for i in range(0,test_num):
  label, file_name_test = file_name_list_test[ i ]
  test_img = cv2.imread(file_name_test)
  #画像をグレーに
  gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  #KAZEで局所特徴点(kp)、局所特徴量を取得(des)
  kp, des = kaze.detectAndCompute(gray_img , None)
  #入力画像のBoVWベクトルを生成
  bowDescriptors = bowExtractor.compute(gray_img, kp)
  #入力画像に対して学習済みSVMラベルを予測
  _,predict = svm.predict(bowDescriptors)
  #出力
  print(i,'枚目predict =' ,int(predict[0][0]))
  if label == int(predict[0][0]):
    correct_num+=1
  if i <=15:
    print('正解 = 0')
  else:
     print('正解 = 1')

print(correct_num,"/",test_num)

犬 = 0 ,猫 = 1
0 枚目predict = 1
正解 = 0
1 枚目predict = 0
正解 = 0
2 枚目predict = 0
正解 = 0
3 枚目predict = 1
正解 = 0
4 枚目predict = 1
正解 = 0
5 枚目predict = 0
正解 = 0
6 枚目predict = 0
正解 = 0
7 枚目predict = 0
正解 = 0
8 枚目predict = 1
正解 = 0
9 枚目predict = 0
正解 = 0
10 枚目predict = 0
正解 = 0
11 枚目predict = 1
正解 = 0
12 枚目predict = 0
正解 = 0
13 枚目predict = 0
正解 = 0
14 枚目predict = 0
正解 = 0
15 枚目predict = 0
正解 = 0
16 枚目predict = 1
正解 = 1
17 枚目predict = 0
正解 = 1
18 枚目predict = 1
正解 = 1
19 枚目predict = 1
正解 = 1
20 枚目predict = 1
正解 = 1
21 枚目predict = 1
正解 = 1
22 枚目predict = 0
正解 = 1
23 枚目predict = 0
正解 = 1
24 枚目predict = 1
正解 = 1
25 枚目predict = 1
正解 = 1
26 枚目predict = 1
正解 = 1
27 枚目predict = 0
正解 = 1
28 枚目predict = 1
正解 = 1
29 枚目predict = 0
正解 = 1
19 / 30
